In [56]:
#Import packages
import pandas as pd

In [58]:
#global values

##Input filepath for the following folders

#Resale flat prices 1990-1999
fp_90_99="Relevant_datasets/ResaleFlatPricesBasedonApprovalDate19901999.csv"
#Resale flat prices 2000-Feb 2012
fp_00_12="Relevant_datasets/ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv"
#Resale flat prices Mar 2012 - Dec 2014
fp_12_14="Relevant_datasets/ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv"
#Resale flat prices Jan 2015-Dec 2016
fp_15_16="Relevant_datasets/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv"
#Resale flat prices Jan 2017 onwards
fp_17_onw="Relevant_datasets/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv"

#Address,coordinates and closest MRT folder
fp_Address_coord="Relevant_datasets/1.4_Address_MRT.csv"

#Resale price index
RPI_fp='Relevant_datasets/Resale_Price_Index_Processed.csv'

#output file path for output folder
output_fp="Relevant_datasets/RSF90onw_wAddress.csv"

In [60]:
#Import Address and coordinates folder
Address=pd.read_csv(fp_Address_coord,index_col=0)

In [62]:
###1.Append all datasets together###
##Read 

#read dataframe 1990-1999
RSF_90_99=pd.read_csv(fp_90_99)

#read dataframe 2000-2012
RSF_00_12=pd.read_csv(fp_00_12)

#read dataframe 2012-2014
RSF_12_14=pd.read_csv(fp_12_14)

#read dataframe 2015-2016
RSF_15_16=pd.read_csv(fp_15_16)

#read dataframe 2017 onwards
RSF_17_onw=pd.read_csv(fp_17_onw)


##Creating list of dataframes for easy reference

RSF_list=[RSF_90_99,RSF_00_12,RSF_12_14,RSF_15_16,RSF_17_onw]

#append all datasets together
RSF90_onw=RSF_90_99
for i in range(1,5):
    RSF90_onw=pd.concat([RSF90_onw, RSF_list[i]])

In [63]:
#merge coordinates and Resale flat dataset together
RSF90_onw=RSF90_onw.merge(Address,left_on='street_name',right_on='street_names_oldver',how='left')

In [64]:
#Expect for Nile road to not be mappable from Address dataset to Resale dataset. To drop these rows
print('All unique street names with no closest MRT identified: ',RSF90_onw[RSF90_onw['Cl_MRT'].isnull()]['street_name_x'].unique())
print('number of rows with Nile road as street name to drop: ',RSF90_onw[RSF90_onw['Cl_MRT'].isnull()]['street_name_x'].count())
#Drop rows with Nile Road
RSF90_onw=RSF90_onw.loc[RSF90_onw['street_name_x']!='NILE RD']

print()
print('After dropping...All unique street names with no closest MRT identified: ',RSF90_onw[RSF90_onw['Cl_MRT'].isnull()]['street_name_x'].unique())
print('After dropping...number of rows with Nile road as street name to drop: ',RSF90_onw[RSF90_onw['Cl_MRT'].isnull()]['street_name_x'].count())

All unique street names with no closest MRT identified:  ['NILE RD']
number of rows with Nile road as street name to drop:  104

After dropping...All unique street names with no closest MRT identified:  []
After dropping...number of rows with Nile road as street name to drop:  0


In [65]:
#Quick check that the dataframe looks proper
RSF90_onw.head()

,month,town,flat_type,block,street_name_x,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,street_name_y,Lat,Lon,street_names_oldver,Coordinates,rLat,rLon,Cl_MRT,Distance_MRT_km
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,ANG MO KIO AVE 1,1.363707,103.847236,ANG MO KIO AVE 1,[103.84723583 1.36370658],0.023801,1.812476,ANG MO KIO MRT STATION,0.738894
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,ANG MO KIO AVE 1,1.363707,103.847236,ANG MO KIO AVE 1,[103.84723583 1.36370658],0.023801,1.812476,ANG MO KIO MRT STATION,0.738894
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,ANG MO KIO AVE 1,1.363707,103.847236,ANG MO KIO AVE 1,[103.84723583 1.36370658],0.023801,1.812476,ANG MO KIO MRT STATION,0.738894
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,ANG MO KIO AVE 1,1.363707,103.847236,ANG MO KIO AVE 1,[103.84723583 1.36370658],0.023801,1.812476,ANG MO KIO MRT STATION,0.738894
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,ANG MO KIO AVE 1,1.363707,103.847236,ANG MO KIO AVE 1,[103.84723583 1.36370658],0.023801,1.812476,ANG MO KIO MRT STATION,0.738894


In [67]:
### To add resale price normalized by year and quarter to make resale prices comparable across time

##Parse Quarter and Year column from 'month' column##

RSF90_onw['month'] = pd.to_datetime(RSF90_onw['month'])
RSF90_onw['Year'] = RSF90_onw['month'].dt.year
RSF90_onw['Quarter'] = RSF90_onw['month'].dt.quarter

#For ease of merging, will create a column of YYYY_Q
RSF90_onw['Yr_Qt']=RSF90_onw.apply(lambda x: str(x['Year'])+"_"+str(x['Quarter']) ,axis=1)


##Import and process Resale Price index##

#import RPI
RPI=pd.read_csv(RPI_fp,index_col=0)

#Realised that Quarter and Years are floats, changing to integer
RPI['Year']=RPI['Year'].astype('str')
RPI['Quarter']=RPI['Quarter'].astype('str')

#For ease of merging, will create a column of YYYY_Q
RPI['Yr_Qt']=RPI.apply(lambda x: str(x['Year'])+"_"+str(x['Quarter']) ,axis=1)


##Merge Resale Price index and Resale Price dataset on year and quarter and create column of real resale flat price##

#Merge on Yr and Qtr
RSF90_onw=RSF90_onw.merge(RPI,how='left',on='Yr_Qt')

#Generate column of real resale price using price index
RSF90_onw['Real_resale_price']=RSF90_onw['resale_price']*100/RSF90_onw['Index']



In [71]:
RSF90_onw

,month,town,flat_type,block,street_name_x,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,rLon,Cl_MRT,Distance_MRT_km,Year_x,Quarter_x,Yr_Qt,Year_y,Quarter_y,Index,Real_resale_price
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,...,1.812476,ANG MO KIO MRT STATION,0.738894,1990,1,1990_1,1990,1,13.058331,68921.518519
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,...,1.812476,ANG MO KIO MRT STATION,0.738894,1990,1,1990_1,1990,1,13.058331,45947.679012
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,...,1.812476,ANG MO KIO MRT STATION,0.738894,1990,1,1990_1,1990,1,13.058331,61263.572016
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,...,1.812476,ANG MO KIO MRT STATION,0.738894,1990,1,1990_1,1990,1,13.058331,45947.679012
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,...,1.812476,ANG MO KIO MRT STATION,0.738894,1990,1,1990_1,1990,1,13.058331,361455.074897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927056,2024-06-01,YISHUN,4 ROOM,426C,YISHUN AVE 11,10 TO 12,92.0,Model A,2015,543000.0,...,1.812431,YISHUN MRT STATION,1.201891,2024,2,2024_2,2024,2,100.000000,543000.000000
927057,2024-06-01,YISHUN,4 ROOM,758,YISHUN ST 72,10 TO 12,84.0,Simplified,1986,528888.0,...,1.812232,YISHUN MRT STATION,0.264632,2024,2,2024_2,2024,2,100.000000,528888.000000
927058,2024-06-01,YISHUN,4 ROOM,844,YISHUN ST 81,10 TO 12,104.0,Model A,1988,590000.0,...,1.812303,KHATIB MRT STATION,0.550464,2024,2,2024_2,2024,2,100.000000,590000.000000
927059,2024-06-01,YISHUN,5 ROOM,509A,YISHUN AVE 4,04 TO 06,113.0,Improved,2018,700000.0,...,1.812362,KHATIB MRT STATION,0.855184,2024,2,2024_2,2024,2,100.000000,700000.000000


In [72]:
RSF90_onw.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name_x', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'remaining_lease', 'street_name_y', 'Lat', 'Lon', 'street_names_oldver',
       'Coordinates', 'rLat', 'rLon', 'Cl_MRT', 'Distance_MRT_km', 'Year_x',
       'Quarter_x', 'Yr_Qt', 'Year_y', 'Quarter_y', 'Index',
       'Real_resale_price'],
      dtype='object')

In [76]:
#rename Multi-generation to Multi generation to ensure consistency
RSF90_onw['flat_type']=RSF90_onw.apply(lambda x:"MULTI GENERATION" if (x['flat_type']=='MULTI-GENERATION') else x['flat_type'],axis=1)

In [80]:
RSF90_onw.to_csv(output_fp)